# Functions and Types for notebook on sparse grids

In [1]:
using Tasmanian
using Random
using Distributions
using IterTools
using DataFrames
using NBInclude

### Types

In [2]:
using Parameters
using Expectations
using Interpolations
using ProgressMeter
using LinearAlgebra
using LaTeXStrings
using BenchmarkTools
using StatsPlots
using Statistics

@with_kw struct Params
    x_min::Float64 = 0.0
    x_max::Float64 = 1.0
    y_min::Float64 = 0.0
    y_max::Float64 = 1.0
    nx::Int64 = 21 # grid points for human capital
    ny::Int64 = 21 # grid points for firm productivity
    nz = 10 #number of draws for the z dimension
    dt::Float64 = 1.0/52.0 # length of a period in years
    r::Float64 = (1.0 + 0.05)^dt - 1.0 # interest rate
    delta::Float64 = 0.0127012273 #job destrution rate
    epsilon::Float64 = 0.001 # distance to stay away from bounds of the interval
    x_grid = collect(range(x_min + epsilon, x_max - epsilon, length = nx))
    y_grid = collect(range(y_min + epsilon, y_max - epsilon, length = nx))
    # log of aggregate shocks are AR(1):
    # ln(zt) = rho*ln(zt-1) + e_k
    # with e_k distributed as N(0,psi^2)
    # and psi = sigma*sqrt(1-rho²)
    rho::Float64 = 0.9997369438 #persitence parameter
    sigma::Float64 = 0.0714488990 #volatility parameter
    psi::Float64 = sigma*sqrt(1.0 - rho^2) #std dev. of innovation term
    distrib_innovation::Any = Normal(0, psi) #0 mean and std = psi
    nb_nodes::Int64 = 10 #number nodes for the expectation
    E::Any = expectation(distrib_innovation, n = nb_nodes) #expectation operator
    nodes_E = Expectations.nodes(E) #nodes for expectation
    weigths_E = weights(E) #weights for expectation
    f0::Float64     =    6.0873503685 # market production parameter
    f1::Float64     =    0.0025488557 # market production parameter 
    f2::Float64     =    2.0529182143 # market production parameter
    f3::Float64     =    -0.1400252578 # market production parameter
    f4::Float64     =    8.0349795180 # market production parameter
    f5::Float64     =    -1.9072145913 # market production parameter
    f6::Float64     =    6.5961298483 # market production parameter
    b0::Float64     = 0.7 # home production parameter
    p_xyz::Function = (x, y, z) -> f0*z*(f1 + f2*x + f3*y + f4*(x^2) + f5*(y^2) + f6*x*y)*dt #value of market production
    b_x::Function = (x) -> b0*p_xyz(x, x, 1.0) #value of market production
    s_xyz::Function = (x,y,z) -> p_xyz(x,y,z) - b_x(x) #surplus
    # VECTORIZED FUNCTIONS
    # input of the form:
    # row: observation
    # column: dimension
    p_xyz_v::Function = x -> f0.*x[:,3].*(f1 .+ f2.*x[:,1] .+ f3.*x[:,2] .+ f4.*(x[:,1].^2) .+ f5.*(x[:,2].^2) .+ f6.*x[:,1].*x[:,2]).*dt #value of market production
    b_x_v::Function = x -> b0.*f0.*1.0.*(f1 .+ f2.*x[:,1] .+ f3.*x[:,1] .+ f4.*(x[:,1].^2) .+ f5.*(x[:,1].^2) .+ f6.*x[:,1].*x[:,1]).*dt #value of market production
    s_xyz_v::Function = x -> p_xyz_v(x) .- b_x_v(x) #surplus
    # VECTORIZED FUNCTIONS
    # input of the form:
    # row: dimension
    # column: observation
    p_xyz_r::Function = x -> f0.*x[3, :].*(f1 .+ f2.*x[1, :] .+ f3.*x[2,:] .+ f4.*(x[1,:].^2) .+ f5.*(x[2,:].^2) .+ f6.*x[1,:].*x[2,:]).*dt #value of market production
    b_x_r::Function = x -> b0.*f0.*1.0.*(f1 .+ f2.*x[1,:] .+ f3.*x[1,:] .+ f4.*(x[1,:].^2) .+ f5.*(x[1,:].^2) .+ f6.*x[1,:].*x[1,:]).*dt #value of market production
    s_xyz_r::Function = x -> p_xyz_r(x) .- b_x_r(x) #surplus
    # GRIDS
    d_log_normal = LogNormal(0.0, sigma)
    perc_z = 0.05
    z_min = quantile(d_log_normal, perc_z)
    z_max = quantile(d_log_normal, 1.0 - perc_z)
    lower_bound = [x_min, y_min, z_min]
    upper_bound = [x_max, y_max, z_max]
    z_grid = collect(range(z_min, z_max, length=nz))
    nodes_xyz = (x_grid, y_grid, z_grid, ) #for package Interpolations

end

Params

In [3]:
function solve_VFI(p::Params; max_iter::Int = 1000, tol::Real = 10^-8, show_every::Int=max_iter)
    
    V_old = zeros(p.nx, p.ny, p.nz)
    V_new = zeros(p.nx, p.ny, p.nz)

    # Initial guess
    for (zIndex, zValue) in enumerate(p.z_grid)
    for (yIndex, yValue) in enumerate(p.y_grid)
        for (xIndex, xValue) in enumerate(p.x_grid)
            V_old[xIndex, yIndex, zIndex] = p.s_xyz(xValue, yValue, zValue) + ((1.0 - p.delta)/(1.0 + p.r))*max(0.0, p.s_xyz(xValue, yValue, zValue))
        end
    end
    end

    itp = interpolate(p.nodes_xyz, V_old, Gridded(Linear()))
    etp = extrapolate(itp, Line())
    V_old_interpolated = (x) -> etp(x[1], x[2], x[3])

    # Initialize
    for i = 1:max_iter

        for (zIndex, zValue) in enumerate(p.z_grid)
            for (yIndex, yValue) in enumerate(p.y_grid)
                for (xIndex, xValue) in enumerate(p.x_grid)
                    V_new[xIndex, yIndex, zIndex] = p.s_xyz(xValue, yValue, zValue) + ((1.0 - p.delta)/(1.0 + p.r)).*sum(p.weigths_E.*[max.(0.0, V_old_interpolated([xValue; yValue; (zValue.^p.rho).*exp.(innovation)])) for innovation in p.nodes_E])
                end
            end
        end

        # DISTANCE
        diff= maximum(abs.(V_new .- V_old))
        if mod(i, show_every) == 0
            println("Iter $(i) Diff : $(diff)")
        end

        if diff < tol
            println("Iter $(i) Convergence reached")
            break
        end

        #UPDATE
        V_old = copy(V_new)
        itp = interpolate(p.nodes_xyz, V_old, Gridded(Linear()))
        etp = extrapolate(itp, Line())
        V_old_interpolated = (x) -> etp(x[1], x[2], x[3])

    end

    # Return value function
    return V_old_interpolated
    
end



solve_VFI (generic function with 1 method)

### Tasmanian.jl

In [4]:
#remove duplicate rows from matrix
function remove_dup(A)
    Matrix(unique(DataFrame(A, :auto)))
end

#Function to create a sparse grid by accumulating successively refined grids
#and getting rid of duplicate points
function create_grid(f::Function; iDim = 2, iOut = 1, iDepth = 4, iOrder = 1, tol = 0.01, domain = [0 1.0; 0 1],
                    K::Int = 5, sRule="localp", sCriteria="classic", max_points = Inf)
    
    # initial grid
    list_grids = []
    tsg = Tasmanian.TasmanianSG(iDim, iOut, iDepth)
    Tasmanian.makeLocalPolynomialGrid!(tsg, iOrder=iOrder, sRule=sRule)
    Tasmanian.setDomainTransform!(tsg, domain)
    # sparse grid points from that object
    spPoints = Tasmanian.getPoints(tsg)
    # values on sparse grid
    spVals = [f(Array(spPoints[i,:])) for i in 1:size(spPoints,1)]
    # load points needed for such values
    Tasmanian.loadNeededPoints!(tsg, spVals)
    push!(list_grids, spPoints)
    
    # refinement loop
    max_nb_points = zeros(Int, 1)
    for k = 1:K
        Tasmanian.setSurplusRefinement!(tsg, tol, sCriteria=sCriteria)
        sparse_points_refined = Tasmanian.getNeededPoints(tsg)
        # Check if more points needed
        if Tasmanian.getNumNeeded(tsg) == 0
            break
        end
        sparse_vals_refined = [f(Array(sparse_points_refined[i,:])) for i in 1:size(sparse_points_refined, 1)]
        # load points needed for such values
        Tasmanian.loadNeededPoints!(tsg, sparse_vals_refined)
        # Append 
        push!(list_grids, sparse_points_refined)
        max_nb_points[1] += size(list_grids[k], 1)
        if max_nb_points[1] >= max_points
            break
        end
    end
        
    final_grid = list_grids[1]
    for i=2:length(list_grids)
        final_grid = vcat(final_grid, list_grids[i])
    end
    
    return(remove_dup(final_grid))
    
end

create_grid (generic function with 1 method)

### AdaptiveSparseGrids

Function to extract the grid

In [5]:
using AdaptiveSparseGrids

function extract_grid(fun)
    
    """
    Function to extract the sparse grid from an AdaptiveSparseGrid object.
    Returns a matrix with row = observation, column = dimension
    """
    nb_dim = length(fun.nodes[first(keys(fun.nodes))].x)
    grid = zeros(length(keys(fun.nodes)), nb_dim)

    for (index_key, key) in enumerate(keys(fun.nodes))
        for i = 1:nb_dim
            grid[index_key, i] = fun.nodes[key].x[i]   
        end
    end
    
    # Need to rescale 
    grid = transpose(fun.bounds[:, 1] .+ transpose(grid).*(fun.bounds[:,2] - fun.bounds[:,1]))
    
    return grid
    
end


extract_grid (generic function with 1 method)